## importing

In [1]:
import os
import tempfile
from datetime import datetime, timedelta
from dateutil import parser
import pandas as pd
import uuid 
import requests
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.timescalevector import TimescaleVector
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from timescale_vector import client
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint as HuggingFaceHub
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel
from operator import itemgetter
from typing import List, Optional, Tuple
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    format_document,
)
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import (
    Runnable,
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from typing import Dict, Any
from dotenv import find_dotenv, load_dotenv

_ = load_dotenv(find_dotenv())
huggingfacehub_api_token = os.environ['HUGGINGFACE_API_KEY']


## testing

In [ ]:
folder_path = "../data/reformatted_transcripts copy"

In [ ]:
# Assuming your files are stored in the specified folder
files = os.listdir(folder_path)
documents = []

for file in files:
    file_path = os.path.join(folder_path, file)
    loader = CSVLoader(file_path, csv_args={"delimiter": ";"}, metadata_columns=["speaker_name", "date_time"])
    documents.extend(loader.load())

# Split the documents into chunks for embedding
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=409,
    chunk_overlap=100,
)
docs = text_splitter.split_documents(documents)

In [ ]:
ids = [uuid.uuid1() for _ in range(len(docs))]  # Generate UUIDs for each document

In [ ]:
def parse_date(date_string: str) -> datetime:
    if date_string is None:
        return None
    time_format = "%Y-%m-%dT%H:%M:%S"
    return datetime.strptime(date_string, time_format)

In [ ]:
dt = parse_date("2021-06-04T00:02:45")

## function

In [ ]:
# Function to take in a date string in the past and return a uuid v1
def create_uuid(date_string: str):
   if date_string is None:
       return None
   time_format = '%a %b %d %H:%M:%S %Y %z'
   datetime_obj = datetime.strptime(date_string, time_format)
   uuid = client.uuid_from_time(datetime_obj)
   return str(uuid)

In [ ]:
def parse_date(date_string: str) -> datetime:
    if date_string is None:
        return None
    time_format = "%a %b %d %H:%M:%S %Y %z"
    return datetime.strptime(date_string, time_format)

In [ ]:
def load_dataset_from_folder(
    folder_path,
    service_url,
    collection_name,
    partition_interval=timedelta(days=7)
):
    # Assuming your files are stored in the specified folder
    files = os.listdir(folder_path)
    documents = []

    for file in files:
        file_path = os.path.join(folder_path, file)
        loader = CSVLoader(file_path, csv_args={"delimiter": ";"}, metadata_columns=["speaker_name", "date_time"])
        documents.extend(loader.load())

    # Split the documents into chunks for embedding
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=409,
        chunk_overlap=100,
    )
    docs = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name = "intfloat/multilingual-e5-large")
        
    # Create a Timescale Vector instance from the collection of documents
    TimescaleVector.from_documents(
        embedding=embeddings,
        ids=[doc.metadata["id"] for doc in docs],
        documents=docs,
        collection_name=collection_name,
        service_url=service_url,
        time_partition_interval=partition_interval,
    )

## try 1

In [ ]:
collection_name = "tryingagaineerdata"
SERVICE_URL = os.environ["timescale_url"]
embeddings = HuggingFaceEmbeddings(model_name = "intfloat/multilingual-e5-large")
partition_interval = timedelta(days=7)

In [ ]:
load_dataset_from_folder(
    folder_path="../data/reformatted_transcripts copy",
    service_url=SERVICE_URL,
    collection_name="tryingagaineerdata",
    partition_interval=timedelta(days=7)
)

In [ ]:
document_content_description = "The content of transcripts from EER meetings"
metadata_field_info = [
    AttributeInfo(
        name="uuid",
        description="A UUID v1 generated from the date",
        type="uuid",
    ),
    AttributeInfo(
        # This is a special attribute represent the timestamp of the uuid.
        name="date_time",
        description="The timestamp in the zoom transcript",
        type="datetime.datetime",
    ),
    AttributeInfo(
        name="speaker_name",
        description="The name of the speaker",
        type="string",
    ),
]

In [ ]:
vectorstore = TimescaleVector(
    embedding=embeddings,
    collection_name=collection_name,
    service_url=SERVICE_URL,
    time_partition_interval=partition_interval,
)

In [ ]:
llm = HuggingFaceHub(
    repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1",
    huggingfacehub_api_token=os.getenv("HUGGINGFACE_API_KEY")
)

In [ ]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
    verbose=True,
)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# RAG chain
chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("What did we talk about in the meeting in 2021?")

In [ ]:
SERVICE_URL = os.environ["timescale_url"]
collection_name = "anotherattempt"
embeddings = HuggingFaceEmbeddings(model_name = "intfloat/multilingual-e5-large")

In [ ]:
load_dataset_from_folder(
    folder_path="../data/reformatted_transcripts copy",
    service_url=SERVICE_URL,
    collection_name="anotherattempt",
    partition_interval=timedelta(days=7)
)

In [ ]:
document_content_description = "The content of transcripts from EER meetings"
metadata_field_info = [
    AttributeInfo(
        name="id",
        description="A UUID v1 generated from the date",
        type="uuid",
    ),
    AttributeInfo(
        # This is a special attribute represent the timestamp of the uuid.
        name="date_time",
        description="The timestamp in the zoom transcript",
        type="datetime.datetime",
    ),
    AttributeInfo(
        name="speaker_name",
        description="The name of the speaker",
        type="string",
    ),
]

In [ ]:
vectorstore = TimescaleVector(
    embedding=embeddings,
    collection_name=collection_name,
    service_url=SERVICE_URL,
    time_partition_interval=partition_interval,
)

In [ ]:
llm = HuggingFaceHub(
    repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1",
    huggingfacehub_api_token=os.getenv("HUGGINGFACE_API_KEY")
)

In [ ]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
    verbose=True,
)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# RAG chain
chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("What did we talk about in the meeting in 2021?")

## try 2

In [ ]:
def create_uuid(date_string: str):
    if date_string is None:
        return None
    time_format = '%a %b %d %H:%M:%S %Y'
    date_string = date_string.strip()
    datetime_obj = datetime.strptime(date_string, time_format)
    uuid = client.uuid_from_time(datetime_obj)
    return str(uuid)

In [ ]:
folder_path = "../data/reformatted_transcripts"
files = os.listdir(folder_path)
documents = []

for file in files:
    file_path = os.path.join(folder_path, file)
    loader = CSVLoader(
        file_path,
        csv_args={"delimiter": ";"},
        metadata_columns=["speaker_name", "date_time"])
    loaded_documents = loader.load()
    
    for doc in loaded_documents:
        date_string = doc.metadata.get("date_time")  # Assuming "date_time" is a metadata key
        if date_string:
            uuid = create_uuid(date_string)
            doc.metadata["uuid"] = uuid

    documents.extend(loaded_documents)

In [ ]:
# Split the documents into chunks for embedding
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=409,
    chunk_overlap=100,
)
docs = text_splitter.split_documents(documents)

In [2]:
# Define collection name
SERVICE_URL = os.environ["timescale_url"]
COLLECTION_NAME = "eerallfiles"
embeddings = HuggingFaceEmbeddings(model_name = "intfloat/multilingual-e5-large")

c:\Users\szh\Desktop\EER\EER-chatbot-UI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Create a Timescale Vector instance from the collection of documents
db = TimescaleVector.from_documents(
     embedding=embeddings,
     ids = [doc.metadata["uuid"] for doc in docs],
     documents=docs,
     collection_name=COLLECTION_NAME,
     service_url=SERVICE_URL,
     time_partition_interval=timedelta(days = 7),)

In [3]:
db = TimescaleVector(
    embedding=embeddings,
    collection_name="eerallfiles",
    service_url=SERVICE_URL
)

In [21]:
start_dt = datetime(2021, 5, 28, 00, 10, 00)
end_dt = datetime(2024, 5, 28, 00, 20, 00)
query = "when was the playing with the sun project first mentioned?"

In [22]:
db.similarity_search_with_score(query, start_date=start_dt, end_date=end_dt, k = 10)

[(Document(page_content='transcript_text: Yeah. I mean, especially the playing with the sun resources site. So', metadata={'row': 278, 'uuid': '2173ea80-0588-11ee-a000-ca85d08d882b', 'source': '../data/reformatted_transcripts\\2023-06-08 09.05.49_rf.csv', 'date_time': 'Thu Jun 08 01:07:53 2023 ', 'speaker_name': 'Amos Bokonen Blanton'}),
  0.14093492735207336),
 (Document(page_content="transcript_text: Well, that's where I'm wondering if you know it. If the reference to it in the chapter of the Kappa about playing with the sun", metadata={'row': 266, 'uuid': 'b9250900-0587-11ee-bc82-4f8fe0752b5e', 'source': '../data/reformatted_transcripts\\2023-06-08 09.05.49_rf.csv', 'date_time': 'Thu Jun 08 01:04:58 2023 ', 'speaker_name': 'Unknown Speaker'}),
  0.14244196243411156),
 (Document(page_content="transcript_text: Yeah, no, I think that's absolutely right. And maybe I can just sort of set the stage as to as to the frame of the research a little bit just so you get a an idea of what I'm pl

In [14]:
llm = HuggingFaceHub(
    repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1",
    huggingfacehub_api_token=os.getenv("HUGGINGFACE_API_KEY")
)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\szh\.cache\huggingface\token
Login successful


In [15]:
prompt = """
    You are a chatbot working for the Experimenting Experiencing Reflecting (EER) Project, a research endeavor investigating the connections between art and science.
    You have access to a vast collection of documents, including research papers, meeting transcripts, and other relevant materials.
    The dialogue from the meetings may contain errors, prompting you to deduce the most probable information from the surrounding context.
    Your main task is to provide information and answer questions based on the available data to assist the project members in their tasks.
    All questions should pertain to the EER Project unless specified otherwise.
    In cases where questions mention "we," it is assumed to be referencing a member of the EER group.
    As the user engaging with you lacks knowledge of the provided context, ensure to provide relevant details for understanding 
    If uncertain about any details, kindly indicate so in your responses and maintain brevity in your answers.

    Context: {context}
    Question: {question}
    Answer: 
    
    """

In [16]:
class SimilarityRetriever(Runnable[Dict[str, Any], Dict[str, Any]]):
    
    def __init__(self, 
            db: TimescaleVector = None,
            embedding_model_name: str = "intfloat/multilingual-e5-large",
            collection_name: str = "eerallfiles",
            timescale_url: str = None):
        if db is None:
            db = TimescaleVector(
                    embedding=HuggingFaceEmbeddings(model_name=embedding_model_name),
                    collection_name=collection_name,
                    service_url=SERVICE_URL
                )
            self.db = db
        else:
            self.db = db
            
    def invoke(self, input: Dict[str, Any], config: Optional[Any] = None) -> Dict[str, Any]:
        query = input.get("question", "")
        start_date = input.get("start_date")
        end_date = input.get("end_date")
        
        # Check if the query is empty or not provided
        if query is None or query == "" or isinstance(query, str) and query.isspace():
            raise ValueError("Invalid or empty query provided")

        # Perform similarity search in database with query, start_date, and end_date
        context = self.db.similarity_search_with_score(query, start_date=start_date, end_date=end_date)

        # Return the context as part of the output dictionary
        return context

In [20]:
rag_chain = (
        {"context": SimilarityRetriever(self.db), "question": RunnablePassthrough()} 
        | PromptTemplate(template=prompt, input_variables=["context", "question"])
        | self.llm
    )


NameError: name 'self' is not defined

In [18]:
rag_chain.invoke({"question": query, "start_date": start_dt, "end_date": end_dt})

TypeError: Serializable.__init__() takes 1 positional argument but 2 were given

In [ ]:
user_question = "What is Peter's last name?"
start_date = datetime(2022, 4, 1)
end_date = datetime(2024, 4, 22)

## by importting

In [1]:
from main import ChatBot, SimilarityRetriever

In [2]:
bot = ChatBot(custom_template="Testing", repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", temperature=1.0)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.


Token is valid (permission: write).
Your token has been saved to C:\Users\szh\.cache\huggingface\token
Login successful


In [3]:
from datetime import datetime
start_date = datetime(2023, 5, 28)
end_date = datetime.now()
input = "how is the group using creativity as a tool?"

In [4]:
input_with_dates = {"question": input, "start_date": start_date, "end_date": end_date}

In [7]:
bot.rag_chain.invoke(input_with_dates)

BadRequestError:  (Request ID: 7zeNwElcw1Rm8u3cejKf4)

Bad request:
Authorization header is correct, but the token seems invalid

In [5]:
# Generate response
SimilarityRetriever().invoke(input_with_dates)

[(Document(page_content="transcript_text: So where do we where does this leave us? Experimenting like crazy, still using available tools? And in parallel, exploring with the Center for Humanities computing, whether we can kind of set up something more locally is isn't that the best?", metadata={'row': 5, 'uuid': 'c958e000-107f-11ee-819e-ea8c702a7ca9', 'source': '../data/reformatted_transcripts\\2023-06-22 09.31.20_rf.csv', 'date_time': 'Thu Jun 22 00:05:52 2023 ', 'speaker_name': 'Andreas Roepstorff'}),
  0.1732974791558759),
 (Document(page_content='transcript_text: creativity in the library. Yep.', metadata={'row': 181, 'uuid': '5894cc80-0585-11ee-8b25-7e7a903bec01', 'source': '../data/reformatted_transcripts\\2023-06-08 09.05.49_rf.csv', 'date_time': 'Thu Jun 08 00:47:57 2023 ', 'speaker_name': 'Andreas Roepstorff'}),
  0.17369975086749911),
 (Document(page_content='transcript_text: So how does EER explore aesthetics?', metadata={'row': 8, 'uuid': '9cfd2000-befa-11ee-8337-69d677b180